In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import os
import requests
from io import BytesIO
from PIL import Image

In [3]:
def build_colorizer_model(input_shape=(256, 256, 1)):
    """
    Builds a simple Keras model for image colorization.
    """
    model = keras.Sequential([
        # Input layer expects a single channel (grayscale) image
        keras.Input(shape=input_shape),

        # First convolutional block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        # Second convolutional block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        # Third convolutional block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),

        # Fourth convolutional block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),

        # Output layer with 2 channels for 'a' and 'b'
        layers.Conv2D(2, (3, 3), activation='tanh', padding='same')
    ], name='colorizer')

    return model

In [4]:
# Load a built-in dataset for demonstration
print("Loading CIFAR-10 dataset...")
(X_train_cifar, _), (_, _) = tf.keras.datasets.cifar10.load_data()

# We'll use a subset of the dataset for faster training
X_train_cifar = X_train_cifar[:2000]

# Preprocess images for the model
X_train = []
y_train = []

print("Preprocessing images...")
for img in X_train_cifar:
    # Convert image to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)

    # Resize to 256x256 for our model input
    lab = cv2.resize(lab, (256, 256))

    # Extract L and ab channels
    l_channel = lab[:, :, 0]
    ab_channels = lab[:, :, 1:]

    # Append to our training lists
    X_train.append(l_channel)
    y_train.append(ab_channels)

Loading CIFAR-10 dataset...
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Preprocessing images...


In [5]:
# Convert to numpy arrays and normalize
X_train = np.array(X_train).astype("float32") / 255.0
y_train = np.array(y_train).astype("float32") / 255.0
y_train = (y_train - 128) / 128 # Normalize ab values to [-1, 1] for tanh output
X_train = np.expand_dims(X_train, axis=-1)

print(f"Prepared {len(X_train)} images for training.")

Prepared 2000 images for training.


In [6]:
# Build the model
model = build_colorizer_model()
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

Model: "colorizer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 128)  │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 256, 256, 2)    │         2,306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,589,186 (6.06 MB)

 Trainable params: 1,589,186 (6.06 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Build the model
model = build_colorizer_model()
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

Model: "colorizer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 256, 256, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 128, 128, 128)  │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 256, 256, 2)    │         2,306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,589,186 (6.06 MB)

 Trainable params: 1,589,186 (6.06 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
print("Starting training...")
model.fit(X_train, y_train, epochs=20, batch_size=4)

Starting training...
Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 45s 90ms/step - loss: 1.6290e-05
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6300e-05
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6286e-05
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6262e-05
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6203e-05
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6209e-05
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6237e-05
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6254e-05
Epoch 9/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6258e-05
Epoch 10/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6262e-05
Epoch 11/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6161e-05
Epoch 12/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6323e-05
Epoch 13/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - loss: 1.6305e-05
Epoch 14/20
500/500 ━━━━━━

In [11]:
# Save the trained weights to a file
model.save_weights('/content/drive/MyDrive/12_video_colorizer/colorizer_weights.weights.h5')
print("Training complete and weights saved to colorizer_weights.h5")

Training complete and weights saved to colorizer_weights.h5


In [12]:
def colorize_frame_tf(model, frame):
    """
    Colorizes a single frame using the TensorFlow model.
    """
    # Convert the frame to grayscale and then to LAB
    lab_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l_channel = lab_frame[:, :, 0]

    # Resize grayscale channel for model input
    scaled_l = cv2.resize(l_channel, (256, 256))
    scaled_l = scaled_l.astype("float32") / 255.0
    scaled_l = np.expand_dims(scaled_l, axis=-1)  # Add channel dimension
    scaled_l = np.expand_dims(scaled_l, axis=0)   # Add batch dimension

    # Predict 'a' and 'b' channels
    ab_predicted = model.predict(scaled_l, verbose=0)[0]

    # Resize the predicted 'a' and 'b' channels to the original frame size
    ab_predicted = cv2.resize(ab_predicted, (frame.shape[1], frame.shape[0]))

    # Merge the L channel with the predicted 'a' and 'b' channels
    colorized_lab = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    colorized_lab[:, :, 0] = l_channel
    colorized_lab[:, :, 1:] = ab_predicted * 127 + 128  # Scale back to 0-255 range

    # Convert the colorized LAB frame back to BGR for display
    colorized_frame = cv2.cvtColor(colorized_lab, cv2.COLOR_LAB2BGR)

    return colorized_frame